In [96]:
import numpy as np

mesh = np.load(r"D:\mesh.npy")
joint = np.load(r"D:\joint.npy")

In [ ]:
import plotly.graph_objects as go

# 첫 번째 프레임 추출
first_frame = mesh[0]  # (6890, 3) 형태

# x, y, z 좌표 추출
x = first_frame[:, 0]
y = first_frame[:, 1]
z = first_frame[:, 2]

# 3D 스캐터 플롯 생성
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=2,
        color='blue',  # 마커 색상
        opacity=0.8
    )
)])

# 레이아웃 설정
fig.update_layout(
    title='First Frame of Motion',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# 플롯 출력
fig.show()


In [ ]:
import plotly.graph_objects as go

# 첫 번째 프레임 추출
first_frame = joint[0]  # (6890, 3) 형태

# x, y, z 좌표 추출
x = first_frame[:, 0]
y = first_frame[:, 1]
z = first_frame[:, 2]

# 3D 스캐터 플롯 생성
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=2,
        color='blue',  # 마커 색상
        opacity=0.8
    )
)])

# 레이아웃 설정
fig.update_layout(
    title='First Frame of Motion',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# 플롯 출력
fig.show()


In [ ]:
import plotly.graph_objects as go

# 첫 번째 프레임 추출
first_frame = joint[0]  # (N, 3) 형태 (N개의 조인트)

# x, y, z 좌표 추출
x = first_frame[:, 0]
y = first_frame[:, 1]
z = first_frame[:, 2]

# 3D 스캐터 플롯 생성 (점 + 라벨)
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='text+markers',  # 조인트 번호와 함께 표시
    text=[str(i) for i in range(len(x))],  # 조인트 번호 추가
    textposition="top center",  # 텍스트 위치 조정
    marker=dict(
        size=5,
        color='blue',  # 마커 색상
        opacity=0.8
    ),
    name="Joints"
))

joint_pairs = [[0, 1], [1, 2], [2, 3], [0, 4], [4, 5], [5, 6], [0, 7], [7, 8], [8, 9], [8, 11], [8, 14], [9, 10], [11, 12], [12, 13], [14, 15], [15, 16]]
# joint_pairs를 따라 선 추가
for pair in joint_pairs:
    idx1, idx2 = pair
    fig.add_trace(go.Scatter3d(
        x=[x[idx1], x[idx2]],
        y=[y[idx1], y[idx2]],
        z=[z[idx1], z[idx2]],
        mode='lines',
        line=dict(color='red', width=3),
        name=f"Bone {idx1}-{idx2}"
    ))

# 레이아웃 설정
fig.update_layout(
    title='First Frame of Motion (Joints & Connections)',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    showlegend=False  # 범례 숨김 (조인트가 많아 복잡해짐)
)

# 플롯 출력
fig.show()


In [ ]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from sklearn.neighbors import kneighbors_graph

def compute_laplacian_eigenvalues(point_cloud, k=10, num_eigen=50):
    # KNN 그래프 생성
    W = kneighbors_graph(point_cloud, k, mode='distance', include_self=False)
    W = 0.5 * (W + W.T)  # 대칭 행렬

    # Degree Matrix D
    D = sp.diags(W.sum(axis=1).A1)

    # Laplacian Matrix L = D - W
    L = D - W

    # Compute the first `num_eigen` eigenvalues
    eigenvalues, _ = spla.eigsh(L, k=num_eigen, which='SM')
    
    return eigenvalues

# Example: Compute eigenvalues for two different point clouds

eigvals_1 = compute_laplacian_eigenvalues(mesh[0])
eigvals_2 = compute_laplacian_eigenvalues(mesh[1])

# 고유값 변화 분석
print("Original Eigenvalues:", eigvals_1[:10])
print("Modified Eigenvalues:", eigvals_2[:10])
print("Eigenvalue Differences:", np.abs(eigvals_1 - eigvals_2))


In [ ]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from sklearn.neighbors import kneighbors_graph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def compute_laplacian_eigen(point_cloud, k=10, num_eigen=5):
    """ 포인트 클라우드에서 라플라시안 고유값 및 고유벡터 계산 """
    W = kneighbors_graph(point_cloud, k, mode='distance', include_self=False)
    W = 0.5 * (W + W.T)  # 대칭 행렬로 변환

    D = sp.diags(W.sum(axis=1).A1)  # Degree Matrix D
    L = D - W  # Laplacian Matrix L

    eigenvalues, eigenvectors = spla.eigsh(L, k=num_eigen, which='SM')
    
    return eigenvalues, eigenvectors

# 1. 원래 포인트 클라우드
# 3. 라플라시안 고유값 및 고유벡터 계산
eigvals_1, eigvecs_1 = compute_laplacian_eigen(mesh[0])
eigvals_2, eigvecs_2 = compute_laplacian_eigen(mesh[1])

# 4. 고유벡터 차이 계산 (두 번째 고유벡터 사용)
eig_diff = np.abs(eigvecs_1[:, 0] - eigvecs_2[:, 0])

# 5. 변형된 영역을 컬러맵으로 시각화
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

sc = ax.scatter(mesh[1][:, 0], mesh[1][:, 1], mesh[1][:, 2], 
                c=eig_diff, cmap='jet', s=5)
plt.colorbar(sc, label="Eigenvector Difference (Deformation)")

ax.set_title("Deformation Visualization")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.show()


In [ ]:
for i in range(1):
    eigvals_1, eigvecs_1 = compute_laplacian_eigen(mesh[0])
    eigvals_2, eigvecs_2 = compute_laplacian_eigen(mesh[1])

    # 4. 고유벡터 차이 계산 (두 번째 고유벡터 사용)
    eig_diff = np.abs(eigvecs_1[:, i] - eigvecs_2[:, i])

    # 5. 변형된 영역을 컬러맵으로 시각화
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    sc = ax.scatter(mesh[1][:, 0], mesh[1][:, 1], mesh[1][:, 2], 
                    c=eigvecs_1[:, i], cmap='jet', s=5)
    plt.colorbar(sc, label="Eigenvector Difference (Deformation)")

    ax.set_title(f"{i} Deformation Visualization")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    plt.show()


In [ ]:
eigvecs_1[:1000, 0] = 1

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(mesh[1][:, 0], mesh[1][:, 1], mesh[1][:, 2], 
                c=eigvecs_1[:, 0], cmap='jet', s=5)
plt.colorbar(sc, label="Eigenvector Difference (Deformation)")

ax.set_title(f"{i} Deformation Visualization")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np

# 일부 고유벡터 값 변경
eigvecs_1[:500, 0] = 1

# 3D 산점도 생성
fig = go.Figure(data=[go.Scatter3d(
    x=mesh[1][:, 0],
    y=mesh[1][:, 1],
    z=mesh[1][:, 2],
    mode='markers',
    marker=dict(
        size=3,  # 포인트 크기
        color=eigvecs_1[:, 0],  # 컬러매핑
        colorscale='jet',  # 컬러맵
        colorbar=dict(title="Eigenvector Value"),  # 컬러바 제목
        opacity=0.8
    )
)])

# 레이아웃 설정 (그래프 크기 확대)
fig.update_layout(
    title=f"{i} Deformation Visualization",
    width=1000,  # 그래프 가로 크기
    height=800,  # 그래프 세로 크기
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z"
    )
)

# 그래프 출력
fig.show()


In [ ]:
mesh.shape

In [98]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

def estimate_curvature_pca(point_cloud, k=10):
    """
    PCA를 이용하여 포인트 클라우드의 곡률(평균 곡률) 근사
    :param point_cloud: (N, 3) 형태의 포인트 클라우드
    :param k: k개의 이웃을 사용하여 곡률 계산
    :return: (N,) 형태의 곡률 값
    """
    nbrs = NearestNeighbors(n_neighbors=k).fit(point_cloud)
    curvature = np.zeros(point_cloud.shape[0])
    
    for i in range(point_cloud.shape[0]):
        _, idx = nbrs.kneighbors([point_cloud[i]])
        neighbors = point_cloud[idx[0]]

        # PCA 수행
        pca = PCA(n_components=3)
        pca.fit(neighbors)

        # 세 번째 고유값이 곡률과 관계됨
        curvature[i] = pca.explained_variance_[2]
    
    return curvature

# 예제 실행
curvature_values = estimate_curvature_pca(mesh[0])


In [ ]:
curvature_values

In [ ]:
curvature_values.std()

In [ ]:
import plotly.graph_objects as go

# 3D 산점도 생성 (곡률 값으로 색상 지정)
fig = go.Figure(data=[go.Scatter3d(
    x=mesh[0][:, 0],  # X 좌표
    y=mesh[0][:, 1],  # Y 좌표
    z=mesh[0][:, 2],  # Z 좌표
    mode='markers',
    marker=dict(
        size=3,  # 점 크기
        color=curvature_values,  # 곡률 값으로 색상 매핑
        colorscale='jet',  # 컬러맵 (곡률이 높을수록 빨간색)
        colorbar=dict(title="Curvature"),  # 컬러바 추가
        opacity=0.8
    )
)])

# 레이아웃 설정 (그래프 크기 조정)
fig.update_layout(
    title="Curvature Visualization",
    width=1000,  # 그래프 가로 크기
    height=800,  # 그래프 세로 크기
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z"
    )
)

# 그래프 출력
fig.show()


In [117]:
def estimate_curvature_laplacian(mesh, k=10, num_eigen=10):
    """
    라플라시안 고유값을 이용하여 곡률 근사
    :param mesh: (N, 3) 형태의 포인트 클라우드
    :param k: k개의 이웃을 사용하여 그래프 라플라시안 계산
    :param num_eigen: 몇 개의 고유값을 사용할지
    :return: (N,) 형태의 곡률 값
    """
    eigvals, eigvecs = compute_laplacian_eigen(mesh, k=k, num_eigen=num_eigen)
    
    # 두 번째 고유값(1-indexed)이 곡률과 연관됨
    return eigvecs[:, 1]

# 예제 실행
curvature_values = estimate_curvature_laplacian(mesh[0])


In [ ]:
import plotly.graph_objects as go

# 3D 산점도 생성 (곡률 값으로 색상 지정)
fig = go.Figure(data=[go.Scatter3d(
    x=mesh[0][:, 0],  # X 좌표
    y=mesh[0][:, 1],  # Y 좌표
    z=mesh[0][:, 2],  # Z 좌표
    mode='markers',
    marker=dict(
        size=3,  # 점 크기
        color=curvature_values,  # 곡률 값으로 색상 매핑
        colorscale='jet',  # 컬러맵 (곡률이 높을수록 빨간색)
        colorbar=dict(title="Curvature"),  # 컬러바 추가
        opacity=0.8
    )
)])

# 레이아웃 설정 (그래프 크기 조정)
fig.update_layout(
    title="Curvature Visualization",
    width=1000,  # 그래프 가로 크기
    height=800,  # 그래프 세로 크기
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z"
    )
)

# 그래프 출력
fig.show()


In [ ]:
curvature_values = estimate_curvature_pca(mesh[0])


In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import cv2

def estimate_curvature_pca(mesh):
    """
    PCA를 사용하여 각 점에 대한 곡률을 추정
    :param mesh: (N, 3) 형태의 포인트 클라우드
    :return: (N,) 형태의 곡률 값
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(mesh)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

    curvatures = np.zeros(len(mesh))
    for i, point in enumerate(mesh):
        neighbors = np.asarray(pcd.points)[i-5:i+5]  # 이웃 점들
        if len(neighbors) < 3:
            continue
        cov_matrix = np.cov(neighbors.T)
        eigvals, _ = np.linalg.eig(cov_matrix)
        curvatures[i] = eigvals.min()  # 최소 고유값을 곡률로 사용
    
    return curvatures

# 동영상 저장 설정
video_filename = r'D:\curvature_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_width = 800
frame_height = 600
out = cv2.VideoWriter(video_filename, fourcc, 30.0, (frame_width, frame_height))

# 동영상 생성
num_frames = len(mesh)  # mesh에서 각 프레임의 수
for i in range(num_frames):
    curvature_values = estimate_curvature_pca(mesh[i])
    
    # Matplotlib로 시각화
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(mesh[i][:, 0], mesh[i][:, 1], mesh[i][:, 2], 
                         c=curvature_values, cmap='jet', s=5)

    plt.colorbar(scatter, ax=ax, label="Curvature")

    ax.set_title(f"Frame {i} - PCA Curvature Visualization")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

    # Matplotlib 그래프를 이미지로 저장
    plt.tight_layout()
    fig.canvas.draw()  # 그리기

    # buffer_rgba()로 이미지를 얻고, RGB로 변환
    img = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (4,))  # RGBA로 변환
    img = img[..., :3]  # RGBA에서 RGB로 변환

    # 동영상에 프레임 추가
    out.write(img)

    # 그래프 리셋
    plt.clf()

# 동영상 파일 저장
out.release()
cv2.destroyAllWindows()


In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import cv2

def estimate_curvature_pca(mesh):
    """
    PCA를 사용하여 각 점에 대한 곡률을 추정
    :param mesh: (N, 3) 형태의 포인트 클라우드
    :return: (N,) 형태의 곡률 값
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(mesh)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

    curvatures = np.zeros(len(mesh))
    for i, point in enumerate(mesh):
        neighbors = np.asarray(pcd.points)[max(0, i-5): min(len(mesh), i+5)]  # 이웃 점들
        if len(neighbors) < 3:
            continue
        cov_matrix = np.cov(neighbors.T)
        eigvals, _ = np.linalg.eig(cov_matrix)
        curvatures[i] = eigvals.min()  # 최소 고유값을 곡률로 사용
    
    return curvatures

# 동영상 저장 설정
video_filename = r'D:\curvature_video_difference.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_width = 800
frame_height = 600
out = cv2.VideoWriter(video_filename, fourcc, 30.0, (frame_width, frame_height))

# 동영상 생성
num_frames = len(mesh)  # mesh에서 각 프레임의 수
prev_curvature_values = None  # 이전 프레임의 곡률 저장

for i in range(num_frames):
    curvature_values = estimate_curvature_pca(mesh[i])
    
    if prev_curvature_values is None:
        delta_curvature = np.zeros_like(curvature_values)  # 첫 프레임은 변화량 없음
    else:
        delta_curvature = np.abs(curvature_values - prev_curvature_values)  # 변화량 계산

    prev_curvature_values = curvature_values  # 현재 곡률을 다음 프레임의 기준으로 저장

    # 변화량이 일정 값 이상이면 빨간색 강조
    threshold = np.percentile(delta_curvature, 90)  # 상위 10% 변화량을 강조
    highlight_color = np.where(delta_curvature > threshold, 'red', 'blue')  # 색상 변경

    # Matplotlib로 시각화
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(mesh[i][:, 0], mesh[i][:, 1], mesh[i][:, 2], 
                         c=delta_curvature, cmap='jet', s=5)

    plt.colorbar(scatter, ax=ax, label="Curvature Change")

    ax.set_title(f"Frame {i} - PCA Curvature Change")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

    # Matplotlib 그래프를 이미지로 저장
    plt.tight_layout()
    fig.canvas.draw()  # 그리기

    # buffer_rgba()로 이미지를 얻고, RGB로 변환
    img = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (4,))  # RGBA로 변환
    img = img[..., :3]  # RGBA에서 RGB로 변환

    # 동영상에 프레임 추가
    out.write(img)

    # 그래프 리셋
    plt.clf()

# 동영상 파일 저장
out.release()
cv2.destroyAllWindows()


In [ ]:
import numpy as np
import open3d as o3d
import plotly.graph_objects as go

def estimate_curvature_pca(mesh):
    """
    PCA를 사용하여 각 점에 대한 곡률을 추정
    :param mesh: (N, 3) 형태의 포인트 클라우드
    :return: (N,) 형태의 곡률 값
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(mesh)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

    curvatures = np.zeros(len(mesh))
    for i, point in enumerate(mesh):
        neighbors = np.asarray(pcd.points)[max(0, i-5): min(len(mesh), i+5)]
        if len(neighbors) < 3:
            continue
        cov_matrix = np.cov(neighbors.T)
        eigvals, _ = np.linalg.eig(cov_matrix)
        curvatures[i] = eigvals.min()
    
    return curvatures

# =========================== #
# 🔹 데이터 준비
# =========================== #
num_frames = len(mesh)  # mesh에서 프레임 수
prev_curvature_values = None
frames = []  # 애니메이션 프레임 저장 리스트

for i in range(num_frames):
    curvature_values = estimate_curvature_pca(mesh[i])
    
    if prev_curvature_values is None:
        delta_curvature = np.zeros_like(curvature_values)  # 첫 프레임은 변화량 없음
    else:
        delta_curvature = np.abs(curvature_values - prev_curvature_values)  # 변화량 계산

    prev_curvature_values = curvature_values  # 현재 곡률 저장

    # 변화량이 상위 10% 이상이면 빨간색, 나머지는 파란색
    threshold = np.percentile(delta_curvature, 90)
    colors = np.where(delta_curvature > threshold, 'red', 'blue')

    # =========================== #
    # 🔹 Plotly 프레임 추가
    # =========================== #
    frames.append(go.Frame(
        data=[go.Scatter3d(
            x=mesh[i][:, 0],
            y=mesh[i][:, 1],
            z=mesh[i][:, 2],
            mode='markers',
            marker=dict(
                size=3,
                color=delta_curvature,
                colorscale='jet',
                colorbar=dict(title="Curvature Change"),
                opacity=0.8
            )
        )],
        name=f"Frame {i}"
    ))

# =========================== #
# 🔹 초기 프레임 설정
# =========================== #
fig = go.Figure(
    data=[go.Scatter3d(
        x=mesh[0][:, 0],
        y=mesh[0][:, 1],
        z=mesh[0][:, 2],
        mode='markers',
        marker=dict(
            size=3,
            color=np.zeros(len(mesh[0])),  # 첫 프레임은 변화 없음
            colorscale='jet',
            colorbar=dict(title="Curvature Change"),
            opacity=0.8
        )
    )],
    frames=frames
)

# =========================== #
# 🔹 레이아웃 설정 (애니메이션 컨트롤 추가)
# =========================== #
fig.update_layout(
    title="3D Curvature Change Visualization",
    width=800,
    height=600,
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z"
    ),
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[
            dict(label="▶ Play",
                 method="animate",
                 args=[None, dict(frame=dict(duration=200, redraw=True),
                                  fromcurrent=True)]),
            dict(label="❚❚ Pause",
                 method="animate",
                 args=[[None], dict(frame=dict(duration=0, redraw=False),
                                    mode="immediate",
                                    fromcurrent=True)])
        ]
    )],
    sliders=[dict(
        steps=[dict(method="animate",
                    args=[[f.name], dict(mode="immediate",
                                         frame=dict(duration=0, redraw=True),
                                         transition=dict(duration=0))],
                    label=f.name) for f in frames],
        active=0
    )]
)

# =========================== #
# 🔹 시각화 실행
# =========================== #
fig.show()
